## --- Day 24: Arithmetic Logic Unit ---

The ALU is a four-dimensional processing unit: it has integer variables w, x, y, and z. These variables all start with the value 0. The ALU also supports six instructions:

- `inp a` - Read an input value and write it to variable a.
- `add a b` - Add the value of a to the value of b, then store the result in variable a.
- `mul a b` - Multiply the value of a by the value of b, then store the result in variable a.
- `div a b` - Divide the value of a by the value of b, truncate the result to an integer, then store the result in variable a. (Here, "truncate" means to round the value toward zero.)
- `mod a b` - Divide the value of a by the value of b, then store the remainder in variable a. (This is also called the modulo operation.)
- `eql a b` - If the value of a and b are equal, then store the value 1 in variable a. Otherwise, store the value 0 in variable a.


__To enable as many submarine features as possible, find the largest valid fourteen-digit model number that contains no 0 digits. What is the largest model number accepted by MONAD?__


In [323]:
class LogicUnit:
    def __init__(self, instructions, input):
        self.instructions = instructions
        self.reg = {var: 0 for var in ["w", "x", "y", "z"]}
        self.input = input if type(input) == list else list(str(input))
        self.log = []

    @classmethod
    def from_list(self, instruction_list, input):
        instructions = [tuple(inst.split()) for inst in instruction_list]

        return LogicUnit(instructions, input)

    @classmethod
    def from_file(self, filename, input):
        with open(filename) as f:
            lines = [line.strip() for line in f.readlines()]

        return LogicUnit.from_list(lines, input)

    def _read_input(self):
        return int(self.input.pop(0))

    def _exec(self, instruction, reg=None, inputs=None):
        """Execute a single given instruction"""
        
        # Use object's register values and input queue if none were provided
        if reg is None:
            reg = self.reg

        if inputs is None:
            inputs = self.input

        # Each instruction has 2 or 3 components, per above
        if len(instruction) == 3:
            action, var, op = instruction
            # The operand can be either a variable or a literal
            op_value = int(reg.get(op, op))
        else:
            action, var = instruction

        if action == "inp":
            reg[var] = int(inputs.pop(0))
        elif action == "add":
            reg[var] += op_value
        elif action == "mul":
            reg[var] *= op_value
        elif action == "div":
            reg[var] //= op_value
        elif action == "mod":
            reg[var] %= op_value
        elif action == "eql":
            reg[var] = 1 if reg[var] == op_value else 0
        else:
            raise ValueError("Illegal argument, halting!")

    def execute(self, verbose=False):
        """Exec all instructions (and keep a log)"""
        if verbose:
            print("*"*5, "EXECUTE", "*"*50)
        
        try:
            for step_no in range(len(self.instructions)):
                instruction = self.instructions[step_no]
                result = self._exec(instruction)
                # self.reg.update(result)
                self.log.append(f"[{step_no}]\t" +
                "\t".join([f"{k}={v}" for k, v in self.reg.items()]) +
                "\t" + "input:" + ",".join(self.input) +
                "\t" + str(instruction)
                )
        except IndexError:
            self.log.append("*****HALTED: input queue empty"+ "*"*50)

        if verbose:
            for line in self.log[-18:]:
                print(line)
            
        return self

    def execute_from(self, start_line, reg, input_digit):
        """
        Using supplied values for registers and a single digit input,
        run instructions from start_line until an inp instruction is read
        """
        if start_line < 0 or start_line >= len(self.instructions):
            return reg

        line = start_line
        in_block = (self.instructions[line][0] == "inp")

        while in_block:
            instruction = self.instructions[line]
            self._exec(instruction, reg, inputs=[input_digit])         
            line += 1

            # Stop at the end of the instruction list or when 
            in_block = (line < len(self.instructions) and self.instructions[line][0] != "inp")

        return reg

    def registers(self):
        """Convenience method to unpack all registers in order"""
        return self.reg["w"], self.reg["x"], self.reg["y"], self.reg["z"]



In [334]:
alu.input = list(str(59692994994998))
alu.execute().reg["z"]


0

In [300]:
# For example, here is an ALU program which takes an input number, negates it, and stores it in x:
program = """
inp x
mul x -1
""".strip().split("\n")

assert -5 == LogicUnit.from_list(program, input="5").execute().reg["x"]

In [280]:
# Here is an ALU program which takes two input numbers, then sets z to 1 if the second input number is
# three times larger than the first input number, or sets z to 0 otherwise:
program = """
inp z
inp x
mul z 3
eql z x
""".strip().split("\n")

assert 1 == LogicUnit.from_list(program, input=39).execute().reg["z"]
assert 0 == LogicUnit.from_list(program, input=28).execute().reg["z"]


In [281]:
# Here is an ALU program which takes a non-negative integer as input, converts it into binary,
# and stores the lowest (1's) bit in z, the second-lowest (2's) bit in y, the third-lowest (4's)
# bit in x, and the fourth-lowest (8's) bit in w:
program = """
inp w
add z w
mod z 2
div w 2
add y w
mod y 2
div w 2
add x w
mod x 2
div w 2
mod w 2
""".strip().split("\n")

assert (1, 0, 1, 0) == LogicUnit.from_list(program, input=[10]).execute().registers()

In [298]:
p1 = LogicUnit.from_file("./inputs/Day24.txt", input=1)
p1.execute(verbose=True)

***** EXECUTE **************************************************
[1]	w=1	x=0	y=0	z=0	input:	('mul', 'x', '0')
[2]	w=1	x=0	y=0	z=0	input:	('add', 'x', 'z')
[3]	w=1	x=0	y=0	z=0	input:	('mod', 'x', '26')
[4]	w=1	x=0	y=0	z=0	input:	('div', 'z', '1')
[5]	w=1	x=14	y=0	z=0	input:	('add', 'x', '14')
[6]	w=1	x=0	y=0	z=0	input:	('eql', 'x', 'w')
[7]	w=1	x=1	y=0	z=0	input:	('eql', 'x', '0')
[8]	w=1	x=1	y=0	z=0	input:	('mul', 'y', '0')
[9]	w=1	x=1	y=25	z=0	input:	('add', 'y', '25')
[10]	w=1	x=1	y=25	z=0	input:	('mul', 'y', 'x')
[11]	w=1	x=1	y=26	z=0	input:	('add', 'y', '1')
[12]	w=1	x=1	y=26	z=0	input:	('mul', 'z', 'y')
[13]	w=1	x=1	y=0	z=0	input:	('mul', 'y', '0')
[14]	w=1	x=1	y=1	z=0	input:	('add', 'y', 'w')
[15]	w=1	x=1	y=13	z=0	input:	('add', 'y', '12')
[16]	w=1	x=1	y=13	z=0	input:	('mul', 'y', 'x')
[17]	w=1	x=1	y=13	z=13	input:	('add', 'z', 'y')
*****HALTED: input queue empty**************************************************


### Program interpretation
- 1 Sets z to `D1 + 12`                 (z after step 1 in range 13..21)
- 2 multiply z * 26, add `D2 + 8`       (z after step 2 in range 347..563)
- 3 z*26, add D3+7                      (z after step 3 in range 9030..14654)
- 4 z*26, add D4+4                      (234785..381017)

- 5 z //= 26, z*=0 if (z4%26)-11 == D5, else z*=26, add either D5+4 or 0
- 6 z *= 26, += d6+1
- 7 z //=26, if z6%26 - 1 == D7, z *= 

- 14 


In [224]:
def find_z(input):
    d = [int(d) for d in str(input)]

    # 1-4 just add
    z = d[0] + 12
    z = z*26 + d[1] + 8
    z = z*26 + d[2] + 7
    z = z*26 + d[3] + 4

    # 5
    x = 0 if z%26 - 11 == d[4] else 1
    z = (z//26) * (25*x+1)
    z += x*(d[4]+4)

    return z

z_vals = [find_z(i) for i in range(11111,100000)]

print(min(z_vals), max(z_vals))


9030 381017


In [327]:
from itertools import product

alu = LogicUnit.from_file("./inputs/Day24.txt", input=None)
blank_reg = {var: 0 for var in ["w", "x", "y", "z"]}
start_line = 0

# Initial possible values of z: only 0
z_max = {0: 0}

while start_line < len(alu.instructions):
    next_z_max = {}
    for input_digit, z_start in product(range(1,10), z_max.keys()):
        reg = blank_reg.copy()
        reg["z"] = z_start
        next_z = alu.execute_from(start_line, reg, input_digit)["z"]

        # Get the previous max and "append" the current digit
        this_perm = z_max[z_start] * 10 + input_digit

        if next_z not in next_z_max:
            next_z_max[next_z] = this_perm
        else:
            next_z_max[next_z] = max(this_perm, next_z_max[next_z])
        
    z_max = next_z_max
    start_line += 18
    print(f"Step {start_line//18}, processed z-vals: {len(z_max)}")


if 0 in z_max:
    print(f"Max for z=0 is {z_max[0]}")
else:
    print(f"No 0 in set :(")

Step 1, processed z-vals: 9
Step 2, processed z-vals: 81
Step 3, processed z-vals: 729
Step 4, processed z-vals: 6561
Step 5, processed z-vals: 7290
Step 6, processed z-vals: 65610
Step 7, processed z-vals: 70713
Step 8, processed z-vals: 636417
Step 9, processed z-vals: 656100
Step 10, processed z-vals: 641844
Step 11, processed z-vals: 678771
Step 12, processed z-vals: 681264
Step 13, processed z-vals: 6131376
Step 14, processed z-vals: 6492690
Max for z=0 is 59692994994998


In [329]:
z_max[0] == 59692994994998

True

## Part 2

What is the smallest model number accepted by MONAD?

In [335]:
alu = LogicUnit.from_file("./inputs/Day24.txt", input=None)
blank_reg = {var: 0 for var in ["w", "x", "y", "z"]}
start_line = 0

# Initial possible values of z: only 0
z_min = {0: 0}

while start_line < len(alu.instructions):
    next_z_min = {}
    for input_digit, z_start in product(range(1,10), z_min.keys()):
        reg = blank_reg.copy()
        reg["z"] = z_start
        next_z = alu.execute_from(start_line, reg, input_digit)["z"]

        # Get the previous min and "append" the current digit
        this_perm = z_min[z_start] * 10 + input_digit

        if next_z not in next_z_min:
            next_z_min[next_z] = this_perm
        else:
            next_z_min[next_z] = min(this_perm, next_z_min[next_z])
        
    z_min = next_z_min
    start_line += 18
    print(f"Step {start_line//18}, processed z-vals: {len(z_min)}")


if 0 in z_min:
    print(f"Min for z=0 is {z_min[0]}")
else:
    print(f"No 0 in set :(")

Step 1, processed z-vals: 9
Step 2, processed z-vals: 81
Step 3, processed z-vals: 729
Step 4, processed z-vals: 6561
Step 5, processed z-vals: 7290
Step 6, processed z-vals: 65610
Step 7, processed z-vals: 70713
Step 8, processed z-vals: 636417
Step 9, processed z-vals: 656100
Step 10, processed z-vals: 641844
Step 11, processed z-vals: 678771
Step 12, processed z-vals: 681264
Step 13, processed z-vals: 6131376
Step 14, processed z-vals: 6492690
Min for z=0 is 16181111641521


In [336]:
z_min[0] == 16181111641521

True